In [1]:
import pandas as pd
from torch.utils.data import Dataset
import albumentations as A

In [2]:
import random
import torch
import glob
import os
import json
import matplotlib.pyplot as plt


In [9]:
def plot_test_metrics(json_filepath, metric_name):
    dataset_type = 'test'
    # Load the metrics from the JSON file
    with open(json_filepath, "r") as file:
        metrics_dict = json.load(file)
    
    # Extract the specified metric for each fold
    folds = sorted(list(metrics_dict.keys()), key=int)  # Ensure the folds are in numerical order
    metric_values = [metrics_dict[fold][dataset_type][metric_name][0] for fold in folds]
    
    # Plot the metric values
    plt.figure(figsize=(10, 6))
    plt.plot(folds, metric_values, marker='o', linestyle='-')
    plt.xlabel("Folds")
    plt.ylabel(f"{metric_name.capitalize()} Score")
    plt.title(f"{metric_name.capitalize()} Score w.r.t Folds for {dataset_type.capitalize()} Dataset")
    plt.xticks(folds)  # This ensures each fold is shown on the x-axis
    plt.grid(True)#, which='both', linestyle='--', linewidth=0.5)
    plt.tight_layout()
    plt.savefig(f'./test_{metric_name}_plot_svg.pdf', dpi=300)
    plt.close()

In [10]:
plot_test_metrics('./plots/Efficientb0/metrics_Efficientb0.json', 'auroc')

In [ ]:
plot_test_metrics('./plots/Efficientb0/metrics_Efficientb0.json', 'auroc')